In [3]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [58]:
%ls "../"

 Volume in drive C has no label.
 Volume Serial Number is F616-071E

 Directory of C:\Users\yuvid\Downloads

08/29/2021  07:00 PM    <DIR>          .
08/29/2021  07:00 PM    <DIR>          ..
08/29/2021  05:38 PM    <DIR>          .ipynb_checkpoints
08/12/2021  11:05 PM         1,891,277 17IshaUpanishad.pdf
08/05/2021  09:05 AM           221,145 7th CSSE (1).pdf
07/06/2021  12:46 AM           221,145 7th CSSE.pdf
08/13/2021  08:47 AM        29,769,994 7th-Std-Marathi-Balbharti-textbook-pdf.pdf
08/07/2021  11:15 PM            56,404 Aadhar address change Receipt  Yuvraj.pdf
08/07/2021  11:49 PM            56,655 Aadhar address change Receipt Mummy.pdf
08/07/2021  11:41 PM            56,468 Aadhar address change Receipt Rushi.pdf
08/12/2021  10:42 PM         1,065,024 Aap Ki Ankhon Mein Kuch.mp3
08/25/2021  11:52 AM         9,003,224 ADE_4.5_Installer.exe
08/12/2021  12:54 PM            53,760 ASSIGNMENT 1 PPLE.doc
06/07/2021  11:21 PM         2,977,034 Book1_AnuLearnsToSave.pdf
10/14/20

In [59]:
train_data_dir = "./train"  
log_data_dir = "../logs"
#validation_data_dir = "/content/drive/My Drive/Project/Lollipop1/val"   
test_data_dir = "./test"

In [29]:
from keras.preprocessing.image import ImageDataGenerator

In [61]:
#train_size = 3883
#test_size = 624
def Kiermany2018(batch_size, input_size, data_dir):
    train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rescale=1.0/255,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
        
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    train_path = train_data_dir
    test_path = test_data_dir

    train_generator = train_datagen.flow_from_directory(
      directory=train_path,
      target_size=(224, 224),
      color_mode="rgb",
      batch_size=20,
      class_mode="categorical",
      shuffle=True,
      seed=42)
    test_generator = test_datagen.flow_from_directory(
      directory=test_path,
      target_size=(224, 224),
      color_mode="rgb",
      batch_size=100,
      class_mode=None,
      shuffle=False,
      seed=42)
    
    classes = ['NORMAL_RESIZED','PNEUMONIA_RESIZED']
    train_batches = train_datagen.flow_from_directory(train_path,target_size=(224,224),color_mode='rgb', classes=classes, batch_size=batch_size,class_mode='categorical')
    test_batches = test_datagen.flow_from_directory(test_path, target_size=(224,224),color_mode='rgb', classes=classes, batch_size=batch_size, class_mode='categorical')

    return train_batches, test_batches

In [62]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Multiply,GlobalAveragePooling2D, Add, Dense, Activation, Maximum, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Lambda, UpSampling2D, DepthwiseConv2D, SeparableConv2D
from keras.optimizers import Adam
from keras.initializers import glorot_uniform

In [63]:
def res_conv(X, filters, base, s):
    F1, F2, F3 = filters
    name_base = base + '/branch'

    ##### Branch1 is the main path and Branch2 is the shortcut path #####
    X_shortcut=X
    
    ##### Branch1 #####
    # First component of Branch1 
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    X= Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_1', kernel_initializer=glorot_uniform(seed=0))(X)
  
    # Second component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_2')(X)
    X = Activation('relu', name=name_base + '1/relu_2')(X)
    X = Conv2D(filters=F2, kernel_size=(2,2), strides=(s,s), padding='same', name=name_base + '1/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_3', kernel_initializer=glorot_uniform(seed=0))(X)
    
    ##### Branch2 ####
    X_shortcut = BatchNormalization(axis=-1, name=name_base + '2/bn_1')(X_shortcut)
    X_shortcut = Activation('relu', name=name_base + '2/relu_1')(X_shortcut)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid', name=name_base + '2/conv_1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    
    # Final step: Add Branch1 and Branch2
    X = Add(name=base + '/Add')([X, X_shortcut])

    return X

In [64]:
def res_identity(X, filters, base):
    
    name_base = base + '/branch'
    
    F1, F2, F3 = filters

    ##### Branch1 is the main path and Branch2 is the shortcut path #####
    
    X_shortcut = X
    
    ##### Branch1 #####
    # First component of Branch1 
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    Shortcut= Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_1', kernel_initializer=glorot_uniform(seed=0))(Shortcut)

    # Second component BranchOut 1
    X1 = BatchNormalization(axis=-1, name=name_base + '1/ConvBn_2')(X)
    X1 = Activation('relu', name=name_base + '1/ConvRelu_2')(X1)
    X1 = Conv2D(filters=F2, kernel_size=(2,2), dilation_rate=(2, 2),strides=(1,1), padding='same', name=name_base + '1/Conv_2', kernel_initializer=glorot_uniform(seed=0))(X1)
    
    # Second component BrancOut 2
    X2 = BatchNormalization(axis=-1, name=name_base + '1/SepBn_2')(X)
    X2 = Activation('relu', name=name_base + '1/SepRelu_2')(X2)
    X2 = SeparableConv2D(filters=F2, kernel_size=(2,2), dilation_rate=(2, 2),strides=(1,1), padding='same', name=name_base + '1/SepConv_2', kernel_initializer=glorot_uniform(seed=0))(X2)
    
    # Second component Add-BranchOut
    X = Add(name=base + '/Add-2branches')([X1, X2])
    
    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_3', kernel_initializer=glorot_uniform(seed=0))(X)    
    
    # Final step: Add Branch1 and the original Input itself
    X = Add(name=base + '/Add')([X_shortcut,X])

    return X

In [65]:
def EncoderDecoder(X, name_base):
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name = name_base + '/Downsample1')(X)
    #X = Conv2D(outgoing_depth, (2,2), strides=(1,1), dilation_rate=(2,2), padding='same', name = name_base + '/DC1', kernel_initializer=glorot_uniform(seed=0))(X)    
    X = UpSampling2D(size=(2, 2),interpolation='bilinear',name = name_base + '/Upsample1')(X)
    X = Activation('sigmoid', name = name_base + '/Activate')(X)
    return X


In [66]:
def RDBI(X, filters, base, number):
    
    for i in range(number):
        X = res_identity(X, filters, base+ '/id_'+str(1+i))
    
    return X

In [67]:
def OpticNet(input_size,num_of_classes):
    input_shape=(input_size, input_size, 3) # Height x Width x Channel
    X_input = Input(input_shape)

    X = Conv2D(64, (7,7), strides=(2,2), padding='same', name ='CONV1', kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis=-1, name ='BN1')(X)
    X = Activation('relu', name ='RELU1')(X)
    X = res_conv(X, [64,64,256], 'RC0', 1)
    # MID 1
    X1 = EncoderDecoder(X, 'EncoderDecoder1')
    X2 = RDBI(X, [32,32,256], 'RDBI1',4)
    X = Multiply(name = 'Mutiply1')([X1,X2])
    X = Add(name = 'Add1')([X,X1,X2])
    X = res_conv(X, [128,128,512], 'RC1', 2)
    # MID 2
    X1 = EncoderDecoder(X, 'EncoderDecoder2')
    X2 = RDBI(X, [64,64,512], 'RDBI2',4)
    X = Multiply(name = 'Mutiply2')([X1,X2])
    X = Add(name = 'Add2')([X,X1,X2])
    X = res_conv(X, [256,256,1024], 'RC2', 2)
    # MID 3
    X1 = EncoderDecoder(X, 'EncoderDecoder3')
    X2 = RDBI(X, [128,128,1024], 'RDBI3',3)
    X = Multiply(name = 'Mutiply3')([X1,X2])
    X = Add(name = 'Add3')([X,X1,X2])
    X = res_conv(X, [512,512,2048], 'RC3', 2)
    # MID 4
    X1 = EncoderDecoder(X, 'EncoderDecoder4')
    X2 = RDBI(X, [256,256,2048], 'RDBI4',3)
    X = Multiply(name = 'Mutiply4')([X1,X2])
    X = Add(name = 'Add4')([X,X1,X2])
    X = GlobalAveragePooling2D(name='global_avg_pool')(X)
    X = Dense(256, name='Dense_1')(X)
    X = Dense(num_of_classes, name='Dense_2')(X)
    X = Activation('softmax', name='classifier')(X)
    model = Model(inputs=X_input, outputs=X, name='')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [68]:
from keras import callbacks

def callback_for_training(tf_log_dir_name='./log/',patience_lr=10,snapshot_name=None):
    cb = [None] * 3
    """
    Tensorboard log callback
    """
    tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    cb[0]= tb
   
    
    """
    Early Stopping callback
    """
    #Uncomment for usage
    # early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto',save_best_only=True)
    # cb.apppend(early_stop)
    
    """
    Model Checkpointer
    """
    if snapshot_name != None:
        checkpointer = callbacks.ModelCheckpoint(filepath="optic-net.{epoch:02d}-{val_accuracy:.2f}.hdf5",
                                verbose=0,
                                monitor='val_accuracy')
    else :
        checkpointer = callbacks.ModelCheckpoint(filepath=snapshot_name+".{epoch:02d}-{val_accuracy:.2f}.hdf5",
                                verbose=0,
                                monitor='val_accuracy')
    cb[1] = checkpointer
    
    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_lr=1e-8, mode='auto')
    cb[2] = reduce_lr_loss
    
    return cb


In [69]:
import matplotlib.pyplot as plt
def  plot_loss_acc(history,snapshot_name=None):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'bo')
    plt.plot(epochs, val_loss, 'g')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    if snapshot_name == None:
        filename= 'OpticNet_loss.png'
    else:
        filename= snapshot_name+'_loss.png'
    plt.savefig(filename)
    plt.show()
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'g')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    if snapshot_name == None:
        filename= 'OpticNet_acc.png'
    else:
        filename= snapshot_name+'_acc.png'
    plt.savefig(filename)
    plt.show() 

In [ ]:
!pip install ipdb
import ipdb

In [ ]:
import matplotlib.pyplot as plt
import time
import keras.backend as K
import gc
import pickle
def train(data_dir,logdir, input_size, dataset, batch_size, epoch,snapshot_name):
    if dataset == 'Kermany2018':
        train_batches, test_batches = Kiermany2018(batch_size, input_size, data_dir)
        num_of_classes = 4
        train_size = 5232
        cb = callback_for_training(tf_log_dir_name=logdir,snapshot_name=snapshot_name)
        test_size = 624
        start_time = time.time()
        model = OpticNet(input_size,num_of_classes)
    #ipdb.set_trace()
        history = model.fit(train_batches, shuffle=True, steps_per_epoch=train_size //batch_size, validation_data=test_batches,validation_steps= test_size//batch_size, epochs=epoch, verbose=1)
        end_time = time.time()
        print("--- Time taken to train : %s hours ---" % ((end_time - start_time)//3600))
    
    if snapshot_name == None :
        model.save('OpticNet.h5')
    else :
        model.save(snapshot_name+'.h5')
    plot_loss_acc(history,snapshot_name)
    
train(train_data_dir,log_data_dir,224, 'Kermany2018',100,20,'Snappy')

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
CONV1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
BN1 (BatchNormalization)        (None, 112, 112, 64) 256         CONV1[0][0]                      
__________________________________________________________________________________________________
RELU1 (Activation)  

Total params: 12,495,492
Trainable params: 12,450,308
Non-trainable params: 45,184
__________________________________________________________________________________________________
Epoch 1/20


In [2]:
!pip install pycm

     |████████████████████████████████| 64 kB 1.9 MB/s 
     |████████████████████████████████| 571 kB 10.7 MB/s 


In [3]:
from sklearn.metrics import confusion_matrix, classification_report
import pycm
import numpy as np
from sklearn.metrics import classification_report


def Weighted_Error(y_true,y_pred):

    matrix = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    print("Confusion Matrix")
    print(matrix)
    print("Classification report")
    print(classification_report(y_true.argmax(axis=1),y_pred.argmax(axis=1)))

def print_metric(y_true,y_pred,weighted_error=False):
  cz = pycm.ConfusionMatrix(actual_vector=y_true.argmax(axis=1), predict_vector=y_pred.argmax(axis=1))
    
    # Accuracy
  acc = cz.Overall_ACC
  print("Average Accuracy : "+str(acc*100)+'%')

    # Specificity
  specificity = cz.TNR
  totalprecision = 0
  for key, value in specificity.items():
    totalprecision = totalprecision + value
    print('Average Specificity : '+str(totalprecision*100/4.0)+'%')

    # Sensitivity
    recall = cz.TPR
    totalrecall = 0
    for key, value in recall.items():
        totalrecall = totalrecall  + value
    print('Average Sensitivity : '+str(totalrecall*100/4.0)+'%')

    if weighted_error==True:
        Weighted_Error(y_true,y_pred)

In [4]:
import tensorflow as tf
import keras
import argparse
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import time

def test(datadir,dataset,weights):

    test_datagen = ImageDataGenerator(rescale=1.0/255)
    image_size = 224

    if dataset=='Kermany2018':
      classes = ['CNV', 'DME','DRUSEN','NORMAL']
      batch=95
      test_batches = test_datagen.flow_from_directory(datadir, target_size=(image_size,image_size),color_mode='rgb', classes=classes, batch_size=batch, class_mode='categorical')

    imgs, y_true = next(test_batches)
    model = load_model('/content/Snappy.h5')
    start= time.time()
    y_pred = model.predict(imgs)
    end = time.time()

    print ((end-start)/1000)
    
    if dataset=='Kermany2018':
      print_metric(y_true,y_pred,weighted_error=True)
test("/content/drive/My Drive/Project/Lollipop1/test",'Kermany2018','/content/Snappy.h5')


Found 0 images belonging to 4 classes.


OSError: ignored